<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-libraries" data-toc-modified-id="Import-libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import libraries</a></span></li><li><span><a href="#Load-spanish-postal-codes" data-toc-modified-id="Load-spanish-postal-codes-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load spanish postal codes</a></span></li><li><span><a href="#Prepare-REGOE-search" data-toc-modified-id="Prepare-REGOE-search-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Prepare REGOE search</a></span><ul class="toc-item"><li><span><a href="#Configure-selenium" data-toc-modified-id="Configure-selenium-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Configure selenium</a></span></li><li><span><a href="#Define-function-to-extract-data-from-ersults-table" data-toc-modified-id="Define-function-to-extract-data-from-ersults-table-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Define function to extract data from ersults table</a></span></li><li><span><a href="#Go-over-postal-codes-list" data-toc-modified-id="Go-over-postal-codes-list-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Go over postal codes list</a></span></li><li><span><a href="#Merge-the-results-and-save" data-toc-modified-id="Merge-the-results-and-save-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Merge the results and save</a></span></li></ul></li><li><span><a href="#Tests" data-toc-modified-id="Tests-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Tests</a></span></li></ul></div>

### Import libraries

In [10]:
import time

In [11]:
import tqdm

In [2]:
import requests

In [3]:
from bs4 import BeautifulSoup

In [4]:
import pandas as pd

### Load spanish postal codes

You can find the list here https://códigospostales.es/listado-de-codigos-postales-de-espana/

In [ ]:
cp = pd.read_csv('./data/listado-de-codigos-postales-de-españa.csv', 
                 encoding='latin-1', sep=';',
                 dtype={'codigopostalid': object})

In [7]:
cp.head()

,provincia,poblacion,codigopostalid
0,Araba/Álava,Alegría-Dulantzi,00240
1,Ávila,Candeleda,00548
2,Araba/Álava,Vitoria-Gasteiz,01001
3,Araba/Álava,Vitoria-Gasteiz,01002
4,Araba/Álava,Vitoria-Gasteiz,01003


### Prepare REGOE search

#### Configure selenium
- Check Web Scraping using Selenium and Python [tutorial](https://www.scrapingbee.com/blog/selenium-python/)

In [7]:
from selenium import webdriver
import geckodriver_autoinstaller



geckodriver_autoinstaller.install()  # Check if the current version of geckodriver exists
                                     # and if it doesn't exist, download it automatically,
                                     # then add geckodriver to path

driver = webdriver.Firefox()

In [6]:
base_url = 'https://servicio.mapama.gob.es/regoe/Publica/Operadores2.aspx'

#### Define function to extract data from ersults table

In [12]:
def get_rows(n):
    tabla = driver.find_elements_by_id("CphCuerpoPagina_GVCertificados")
    rows = tabla[0].find_elements_by_xpath("//*[@class='GridViewRowEsilo' or @class='GridViewAlternatingRowEstilo']")
    df = pd.DataFrame([[td.text for td in row.find_elements_by_xpath(".//td")] for row in rows])
    try:
        links = tabla[0].find_element_by_class_name("GridViewPaginacion").find_elements_by_xpath(".//a")
        num = len(links)
        links[n].click()
    except:
        pass
        
    return df

#### Go over postal codes list

In [ ]:
total_pages = 0
total_records = 0
pages = []
for cod in tqdm.tqdm(cp['codigopostalid'].unique()):
    driver.get(base_url)
    cp_input = driver.find_element_by_name("ctl00$CphCuerpoPagina$txtCodigoPostal")
    cp_input.send_keys(cod)
    driver.find_element_by_id("CphCuerpoPagina_btnBuscar").click()
    # time.sleep(1)
    tabla = driver.find_elements_by_id("CphCuerpoPagina_GVCertificados")
    if tabla:
        
        try:
            links = tabla[0].find_element_by_class_name("GridViewPaginacion").find_elements_by_xpath(".//a")
            num = len(links)
        except:
            num = 1
           
        for i in range(num):
            df = get_rows(i)
            pages.append(df)
            
        total_pages +=num
        print(f'cp:{cod}: {num} pages')

print(f'{total_pages} pages')



#### Merge the results and save

In [275]:
df = pd.concat(pages)

In [282]:
df.drop([0,1], axis=1, inplace=True)

In [284]:
df.columns = ['Razón social','Dirección sede social','Provincia','Comunidad Autónoma','Actividad','Ubicación actividad','Grupo de productos','Entidad de control']

In [285]:
df.to_csv('REGOE_2021.csv')